# FastAPI - Concurrency and async/await

**Source:** [FastAPI Official Docs - Concurrency and async / await](https://fastapi.tiangolo.com/async/)

Understanding async/await is crucial for FastAPI. This notebook covers both the FastAPI-specific guidance and the fundamental async concepts you need to know.

---

## Quick Reference (TL;DR from FastAPI Docs)

### When to use `async def`:

```python
# ✅ Third party library supports await
@app.get('/')
async def read_results():
    results = await some_library()
    return results
```

### When to use regular `def`:

```python
# ✅ Third party library does NOT support await (most database libraries)
@app.get('/')
def results():
    results = some_library()
    return results
```

### General Rule:

- **Use `async def`** if you need to use `await` inside
- **Use `def`** if you're calling synchronous libraries
- **When in doubt**, use regular `def`
- You can **mix both** in the same app - FastAPI handles both correctly

**Important:** You can only use `await` inside functions created with `async def`

## Part 1: Fundamental Concepts

### What is Async Code?

**Asynchronous code** means the language has a way to tell the program that at some point, it will have to **wait for something else** to finish.

During that waiting time, the program can go do **other work** instead of just blocking.

### I/O Bound Operations

This "waiting" typically refers to **I/O operations** (relatively slow compared to CPU/RAM):
- Network requests (HTTP calls, database queries)
- File system operations (reading/writing files)
- User input
- Remote API calls

These are called **"I/O bound"** operations because execution time is consumed mostly by waiting for I/O.

## Part 2: The Critical Behavior - Calling vs Running

### The Fundamental Truth

**Calling an `async def` function ≠ Running its code**

When you call an `async def` function:
1. It **immediately returns a coroutine object**
2. It does **NOT execute** the function body
3. The coroutine must be **scheduled** to actually run

In [ ]:
import asyncio

async def my_async_func():
    print("This won't print until scheduled!")
    return "result"

# Calling it just creates a coroutine - nothing prints!
coro = my_async_func()
print(f"Type: {type(coro)}")
print(f"Value: {coro}")

In [ ]:
# Now actually run it
result = await my_async_func()
print(f"Result: {result}")

### Sync vs Async Execution Timing

In [ ]:
import asyncio

def sync_function():
    print("Sync: This runs immediately when called")
    return "sync result"

async def async_function():
    print("Async: This only runs when scheduled")
    return "async result"

# Sync function - body runs NOW
print("Calling sync function...")
result1 = sync_function()
print(f"Got: {result1}\n")

# Async function - returns coroutine, body doesn't run
print("Calling async function...")
coro = async_function()
print(f"Got: {coro}")
print("(Notice: the print inside async_function didn't happen!)\n")

# Now schedule it
print("Scheduling async function...")
result2 = await coro
print(f"Got: {result2}")

### Comparison Table

| Action | Sync Function (`def`) | Async Function (`async def`) |
|--------|----------------------|-----------------------------|
| **Call the function** | Body executes immediately | Returns coroutine object |
| **What you get back** | The actual return value | A coroutine (not the value) |
| **When body runs** | Right now | When scheduled later |
| **Need event loop** | No | Yes |

## Part 3: Understanding Coroutines

### What is a Coroutine?

**Coroutine** = The fancy term for the object returned by an `async def` function.

It's something that:
- Can start and end like a function
- Can be **paused** ⏸ internally (at `await` statements)
- Can **resume** ⏯ later
- Must be scheduled by an event loop to run

### Under the Hood: How Coroutines Work

Coroutines are built on **generators** (functions with `yield`).

In [ ]:
# Basic generator example
def my_coroutine():
    print("Start")
    yield "WAIT_IO"  # Pause here
    print("Resume after I/O")

coro = my_coroutine()
print(f"Type: {type(coro)}")

# Start execution until first yield
value = next(coro)
print(f"Yielded: {value}")

# Resume (will print "Resume after I/O")
try:
    next(coro)
except StopIteration:
    print("Coroutine finished")

### Simplified Event Loop Simulation

In [ ]:
# Awaitable object (represents an async operation)
class Awaitable:
    def __init__(self, name):
        self.name = name
        self.done = False
        self.result = None

# A coroutine that yields an awaitable
def handler():
    print("Handler start")
    result = yield Awaitable("DB_REQUEST")  # Pause, wait for I/O
    print(f"Got result: {result}")

# Simple event loop
class EventLoop:
    def __init__(self):
        self.ready = []  # Tasks ready to run
        self.waiting = {}  # Tasks waiting for I/O
    
    def create_task(self, coro):
        self.ready.append(coro)
    
    def run(self):
        while self.ready or self.waiting:
            # Run ready tasks
            while self.ready:
                coro = self.ready.pop(0)
                try:
                    awaited = coro.send(None)  # Resume coroutine
                    self.waiting[awaited] = coro  # Mark as waiting
                except StopIteration:
                    pass  # Coroutine finished
            
            # Simulate I/O completion
            for awaited in list(self.waiting):
                print(f"I/O finished for {awaited.name}")
                awaited.done = True
                awaited.result = f"result of {awaited.name}"
                coro = self.waiting.pop(awaited)
                self.ready.append(coro)

# Run multiple handlers concurrently
loop = EventLoop()
loop.create_task(handler())
loop.create_task(handler())
loop.create_task(handler())
loop.run()

## Part 4: The `async` and `await` Syntax

### Modern Python Syntax

Modern Python makes async code look like normal sequential code:

In [ ]:
import asyncio

# Define async function
async def get_burgers(number: int):
    print(f"Preparing {number} burgers...")
    await asyncio.sleep(1)  # Simulate I/O wait
    return [f"Burger {i+1}" for i in range(number)]

# Call it with await
burgers = await get_burgers(2)
print(f"Got: {burgers}")

### Key Points:

1. **`await` tells Python to wait** for the operation to complete
2. During the wait, Python can **do other work**
3. `await` can only be used inside `async def` functions
4. Functions with `async def` must be **awaited** when called

### The Egg and Chicken Problem

If `async def` functions must be awaited, and `await` only works in `async def` functions, how do we start?

**In Jupyter/IPython:** Top-level `await` is supported

**In regular Python scripts:** Use `asyncio.run()` for the entry point

In [ ]:
# This is how you'd do it in a .py file (don't run this in Jupyter)
# import asyncio
#
# async def main():
#     burgers = await get_burgers(2)
#     print(burgers)
#
# if __name__ == "__main__":
#     asyncio.run(main())  # Entry point

# In Jupyter, we can just use top-level await:
burgers = await get_burgers(3)
print(burgers)

## Part 5: Concurrency vs Parallelism

### Concurrency (Async)

**Definition:** Switching between multiple tasks during waiting periods

**Burger analogy:** You order burgers, get a number, sit with your crush, and come back when ready. While waiting, you can do other things (talk, flirt).

**Technical:** One processor switching between tasks during I/O waits

### Parallelism

**Definition:** Multiple processors doing work simultaneously

**Burger analogy:** 8 cashiers all preparing burgers at the same time

**Technical:** Multiple processors/cores running code at the same time

### When to Use Each

#### Concurrency (Async) is Better For:
- **I/O bound** operations (web requests, database queries, file I/O)
- Web applications (lots of waiting for network)
- Many users with slow connections

#### Parallelism is Better For:
- **CPU bound** operations (heavy computations)
- Image/audio processing
- Machine learning training
- Complex math calculations

#### FastAPI Advantage:
You can use **BOTH** - async for web handling + multiprocessing for CPU-intensive tasks

## Part 6: Scheduling Mechanisms

### Three Ways to Schedule Coroutines:

1. **`await`** - Sequential execution (wait for one thing at a time)
2. **`asyncio.create_task()`** - Concurrent execution (run multiple things)
3. **`asyncio.run()`** - Entry point (start the event loop)

### Method 1: `await` - Sequential Execution

In [ ]:
import asyncio
import time

async def task1():
    print("Task 1 starting...")
    await asyncio.sleep(1)
    print("Task 1 done")
    return "Task 1 result"

async def task2():
    print("Task 2 starting...")
    await asyncio.sleep(1)
    print("Task 2 done")
    return "Task 2 result"

# Sequential execution with await
start = time.time()
result1 = await task1()  # Wait for task1 to complete
result2 = await task2()  # Then wait for task2
elapsed = time.time() - start

print(f"\nResults: {result1}, {result2}")
print(f"Total time: {elapsed:.2f} seconds (should be ~2 seconds)")

### Method 2: `asyncio.create_task()` - Concurrent Execution

In [ ]:
import asyncio
import time

async def task1():
    print("Task 1 starting...")
    await asyncio.sleep(1)
    print("Task 1 done")
    return "Task 1 result"

async def task2():
    print("Task 2 starting...")
    await asyncio.sleep(1)
    print("Task 2 done")
    return "Task 2 result"

# Concurrent execution with create_task
start = time.time()

# Schedule both tasks to run concurrently
t1 = asyncio.create_task(task1())
t2 = asyncio.create_task(task2())

# Wait for both to complete
result1 = await t1
result2 = await t2

elapsed = time.time() - start

print(f"\nResults: {result1}, {result2}")
print(f"Total time: {elapsed:.2f} seconds (should be ~1 second!)")

### The Difference is Critical!

- **Without `create_task()`**: Tasks run one after another (2 seconds)
- **With `create_task()`**: Tasks run concurrently (1 second)

This is the power of async - multiple I/O operations at the same time!

## Part 7: Essential asyncio Tools

### `asyncio.gather()` - Run Multiple Coroutines

In [ ]:
import asyncio

async def fetch_user(user_id: int):
    print(f"Fetching user {user_id}...")
    await asyncio.sleep(1)
    return f"User {user_id} data"

# Run multiple coroutines concurrently
results = await asyncio.gather(
    fetch_user(1),
    fetch_user(2),
    fetch_user(3)
)

print(f"\nResults: {results}")
print("All three fetches happened concurrently (~1 second total)")

### `asyncio.wait_for()` - Timeout Protection

In [ ]:
import asyncio

async def slow_operation():
    print("Starting slow operation...")
    await asyncio.sleep(5)
    return "Done!"

# Try with 2 second timeout
try:
    result = await asyncio.wait_for(slow_operation(), timeout=2.0)
    print(result)
except asyncio.TimeoutError:
    print("Operation timed out! (as expected)")

### `asyncio.sleep()` vs `time.sleep()`

**CRITICAL DIFFERENCE:**

In [ ]:
import asyncio
import time

# ❌ BAD - Blocks the event loop
async def bad_sleep():
    print("Bad sleep: Blocking event loop for 1 second...")
    time.sleep(1)  # This BLOCKS everything!
    print("Bad sleep done")

# ✅ GOOD - Yields control during sleep
async def good_sleep():
    print("Good sleep: Yielding during sleep...")
    await asyncio.sleep(1)  # Event loop can run other tasks
    print("Good sleep done")

# Demo the difference
print("Running bad_sleep (blocks):")
await bad_sleep()

print("\nRunning good_sleep (yields):")
await good_sleep()

## Part 8: The Critical Truth About async Without await

### The Problem: Async in Name Only

In [ ]:
import asyncio
import time

# This is async but doesn't yield!
async def fake_async_work():
    print("Doing synchronous work in async function...")
    # No await = no yielding = blocks event loop
    result = sum(range(10000000))  # CPU work
    return result

# Even though we await it, it still blocks
print("Starting fake async work...")
result = await fake_async_work()
print(f"Result: {result}")
print("During the computation above, NO other async tasks could run!")

### The Rule:

**Without `await` inside the function, there's NO yielding to the event loop.**

Even if:
- The function is defined as `async def`
- You call it with `await`

If there's no `await` inside, it runs synchronously and **blocks the event loop**.

### FastAPI Example: The Wrong Way

In [ ]:
# Simulating FastAPI pattern

def some_sync_library():
    """Pretend this is requests.get() or similar"""
    time.sleep(2)  # Blocking I/O
    return "data from sync library"

# ❌ BAD - async wrapper around sync code
async def no_await():
    # No await here = no benefit from async
    result = some_sync_library()  # This BLOCKS!
    return result

# In FastAPI route:
# @app.get("/")
# async def root():
#     result = await no_await()  # Still blocks the event loop!
#     return {"message": result}

print("This pattern defeats the purpose of async!")
print("Other requests cannot be processed while waiting.")

### FastAPI Example: The Right Way

In [ ]:
import asyncio

async def some_async_library():
    """Pretend this is httpx.AsyncClient or similar"""
    await asyncio.sleep(2)  # Non-blocking I/O
    return "data from async library"

# ✅ GOOD - Actually uses await
async def with_await():
    result = await some_async_library()  # Yields during I/O!
    return result

# In FastAPI route:
# @app.get("/")
# async def root():
#     result = await with_await()  # Other requests can run during I/O
#     return {"message": result}

print("This is the correct async pattern!")
result = await with_await()
print(f"Got: {result}")

## Part 9: FastAPI-Specific Guidance

### Path Operation Functions

FastAPI handles both `def` and `async def` intelligently:

#### With `async def`:
```python
@app.get("/")
async def read_items():
    results = await database.fetch_all(query)
    return results
```
- Called directly by FastAPI
- Must use `await` for async operations
- Best for I/O-bound operations

#### With `def`:
```python
@app.get("/")
def read_items():
    results = database.fetch_all(query)  # Sync library
    return results
```
- Run in external threadpool by FastAPI
- Won't block the event loop
- Best when using sync libraries

### Dependencies

Same rules apply to dependencies:

In [ ]:
# Async dependency
async def get_async_db():
    db = await create_async_connection()
    try:
        yield db
    finally:
        await db.close()

# Sync dependency
def get_sync_db():
    db = create_sync_connection()
    try:
        yield db
    finally:
        db.close()

# Usage:
# @app.get("/users")
# async def get_users(db = Depends(get_async_db)):
#     return await db.fetch_all("SELECT * FROM users")

print("FastAPI will handle both dependency types correctly")

### When FastAPI Uses Threadpool

FastAPI automatically runs `def` functions in a threadpool:
- Path operation functions with `def`
- Dependencies with `def`

This prevents blocking the event loop when using sync libraries.

## Part 10: Common Pitfalls and Solutions

### Pitfall 1: Forgetting to Await

In [ ]:
import asyncio

async def get_data():
    await asyncio.sleep(1)
    return {"key": "value"}

# ❌ Wrong - forgot to await
async def bad_handler():
    data = get_data()  # Returns coroutine, not data!
    # This will fail:
    # print(data["key"])  # TypeError: 'coroutine' object is not subscriptable
    print(f"Type: {type(data)}")
    return data

# ✅ Correct - using await
async def good_handler():
    data = await get_data()  # Actually get the data
    print(f"Type: {type(data)}")
    print(f"Value: {data['key']}")
    return data

await bad_handler()
print()
await good_handler()

### Pitfall 2: Mixing Sync and Async Libraries

In [ ]:
# ❌ Wrong - using sync library in async code
# import requests  # Sync library
# 
# async def fetch_data():
#     response = requests.get(url)  # BLOCKS event loop!
#     return response.json()

# ✅ Correct - using async library
# import httpx  # Async library
# 
# async def fetch_data():
#     async with httpx.AsyncClient() as client:
#         response = await client.get(url)  # Non-blocking!
#     return response.json()

print("Use async libraries (httpx, aiohttp) instead of sync ones (requests)")

### Pitfall 3: Nested Event Loops

In [ ]:
import asyncio

async def helper():
    await asyncio.sleep(0.1)
    return 42

# ❌ Wrong - can't use asyncio.run() inside async function
async def bad_caller():
    # result = asyncio.run(helper())  # RuntimeError: cannot run event loop
    pass

# ✅ Correct - just use await
async def good_caller():
    result = await helper()
    print(f"Result: {result}")
    return result

await good_caller()

### Pitfall 4: CPU-Bound Work in Async

In [ ]:
import asyncio

# ❌ Wrong - CPU-bound work blocks event loop
async def heavy_computation():
    # No await = blocks event loop during computation
    result = sum(i*i for i in range(10000000))
    return result

# ✅ Better - offload to executor
import concurrent.futures

def cpu_heavy():
    return sum(i*i for i in range(10000000))

async def good_computation():
    loop = asyncio.get_event_loop()
    # Run in separate thread/process
    result = await loop.run_in_executor(None, cpu_heavy)
    return result

print("For CPU-bound work, use executors or multiprocessing")

## Part 11: Advanced Patterns

### Task Groups (Python 3.11+)

In [ ]:
import asyncio
import sys

if sys.version_info >= (3, 11):
    async def worker(name: str, delay: float):
        print(f"{name} starting...")
        await asyncio.sleep(delay)
        print(f"{name} done")
        return f"{name} result"
    
    # Using task groups for better error handling
    async def run_with_task_group():
        async with asyncio.TaskGroup() as tg:
            task1 = tg.create_task(worker("Worker 1", 1))
            task2 = tg.create_task(worker("Worker 2", 0.5))
            task3 = tg.create_task(worker("Worker 3", 1.5))
        
        # All tasks complete before we get here
        print(f"All done! Results: {task1.result()}, {task2.result()}, {task3.result()}")
    
    await run_with_task_group()
else:
    print("Task groups require Python 3.11+")

### Background Tasks

In [ ]:
import asyncio

async def background_work(name: str):
    print(f"{name}: Starting background work...")
    await asyncio.sleep(2)
    print(f"{name}: Background work done")

async def main_task():
    # Start background task
    task = asyncio.create_task(background_work("BG"))
    
    # Do other work
    print("Main: Doing main work...")
    await asyncio.sleep(1)
    print("Main: Main work done")
    
    # Optionally wait for background task
    await task

await main_task()

### Cancellation

In [ ]:
import asyncio

async def cancellable_task():
    try:
        print("Task starting...")
        await asyncio.sleep(10)
        print("Task completed")
    except asyncio.CancelledError:
        print("Task was cancelled!")
        raise  # Re-raise to properly cancel

async def demo_cancellation():
    task = asyncio.create_task(cancellable_task())
    
    # Wait a bit
    await asyncio.sleep(1)
    
    # Cancel the task
    task.cancel()
    
    try:
        await task
    except asyncio.CancelledError:
        print("Caught cancellation in main")

await demo_cancellation()

## Part 12: Decision Flow Chart

```
Are you using a library that requires await?
│
├─ YES → Use async def + await
│         Example: httpx, databases, motor
│
└─ NO → Is the library synchronous?
    │
    ├─ YES → Use def (FastAPI runs it in threadpool)
    │         Example: requests, psycopg2, SQLAlchemy
    │
    └─ NO I/O at all → Can use either
              │
              ├─ CPU-heavy → Use def + executor
              └─ Simple logic → Use async def
```

## Summary: Key Takeaways

### The Fundamentals:
1. **Calling `async def` creates a coroutine** - it doesn't run the code
2. **Coroutines must be scheduled** via `await`, `create_task()`, or `asyncio.run()`
3. **`await` yields control** to the event loop during I/O waits
4. **Without `await` inside**, no yielding happens even if the function is `async def`

### For FastAPI:
1. **Use `async def`** when calling libraries that support `await`
2. **Use `def`** when calling synchronous libraries (most databases)
3. **Mix freely** - FastAPI handles both correctly
4. **When in doubt** - use `def`

### Performance:
1. **Async shines** for I/O-bound operations (web apps)
2. **Concurrency ≠ Parallelism** - understand the difference
3. **Use `create_task()`** for concurrent I/O operations
4. **CPU-bound work** needs executors or multiprocessing

### Common Mistakes:
1. ❌ Using `time.sleep()` instead of `await asyncio.sleep()`
2. ❌ Forgetting to `await` async functions
3. ❌ Using sync libraries (requests) in async code
4. ❌ Wrapping sync code in `async def` without `await`

---

## Additional Resources

- [FastAPI Async Documentation](https://fastapi.tiangolo.com/async/)
- [Python asyncio Documentation](https://docs.python.org/3/library/asyncio.html)
- [AnyIO Documentation](https://anyio.readthedocs.io/)
- [Asyncer (by FastAPI creator)](https://asyncer.tiangolo.com/)

## Practice Exercises

Try these to solidify your understanding:

In [ ]:
# Exercise 1: Create an async function that fetches data from 3 APIs concurrently
import asyncio

async def fetch_api(api_num: int):
    await asyncio.sleep(1)  # Simulate API call
    return {"api": api_num, "data": f"Response from API {api_num}"}

async def fetch_all_apis():
    # Your code here - use create_task or gather
    pass

# Test it
# results = await fetch_all_apis()
# print(results)

In [ ]:
# Exercise 2: Implement timeout for a slow operation
async def very_slow_operation():
    await asyncio.sleep(10)
    return "Finally done!"

async def fetch_with_timeout():
    # Your code here - add 2 second timeout
    pass

# Test it
# await fetch_with_timeout()